In [32]:
"""문장 단위로 relevant span 내에 문장이 들어가는지 여부를 labeling 해주는 notebook"""

import json

split = 'train'
data_path = './data/ALL/jsonl/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
n_meetings = len(data)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 162 meetings in the train set.


In [33]:
from nltk import word_tokenize
# tokneize a sent
# 문장을 단어 단위로 쪼개는 느낌..!
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [34]:
# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [35]:
from tqdm import tqdm
import pandas as pd

df= pd.DataFrame(columns= ['document_id', 'query', 'sentence', 'relevant_idx', 'sentence_idx', 'label'])


specific_data = []
for i in tqdm(range(len(data))):

    for j in range(len(data[i]['specific_query_list'])):
        query = tokenize(data[i]['specific_query_list'][j]['query'])
        target = tokenize(data[i]['specific_query_list'][j]['answer']) 
        rel_span= [[int(start), int(end)] for start, end in data[i]['specific_query_list'][j]['relevant_text_span']]

        for k in range(len(data[i]['meeting_transcripts'])):
            cur = dict()
            for start, end in rel_span:
                if start<= k+1 <= end:
                    label= 1
                    break;
                else: label= 0

            cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ' : ' # 화자
            cur_turn = clean_data(cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content']))


            
            cur['query'] = clean_data(query)
            cur['sentence']= cur_turn
            cur['document_id']= i
            cur['label']= label
            cur['relevant_idx']= rel_span
            cur['sentence_idx']= k

            specific_data.append(cur)
        
print('Total {} query-summary pairs in the {} set'.format(len(specific_data), split))
# print(specific_data[2])
with open('./data/dpr_' + split + '.jsonl', 'w') as f:
    for i in range(len(specific_data)):
        print(json.dumps(specific_data[i]), file=f)

100%|██████████| 162/162 [01:41<00:00,  1.60it/s]


Total 538174 query-summary pairs in the train set
